<a href="https://colab.research.google.com/github/JaspreetKhela/amazon-vine-analysis/blob/main/vine_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and initiate Spark; this is boilerplate code

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Import the dataset

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

Transform the dataset

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
# Dropping null values to have a clean data for review 
clean_df = df.dropna()
clean_df.show()

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

In [ ]:
# Using filter for total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show()

In [ ]:
# total_votes_df Size
total_votes_df.count()

In [ ]:
# helpful_votes_df Size
helpful_votes_df.count()

In [ ]:
# Using filter where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
paid_df.show()

In [ ]:
# Using filter where a review was written as not a part of the Vine program (unpaid), vine == 'N'.
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
unpaid_df.show()

In [ ]:
# The total number of reviews for Vine members
total_vine_reviews = paid_df.count()

# The number of 5-star reviews for all Vine members
fivestar_vine_reviews = paid_df.filter(df['star_rating'] == 5).count()

#  percentage of 5-star reviews  for all Vine members
per_fivestar_vine_reviews = fivestar_vine_reviews/total_vine_reviews * 100
print(f'The total number of reviews for Vine members is : {total_vine_reviews}.\n \
The number of 5-star reviews for all Vine members: {fivestar_vine_reviews}.\n  \
The percentage of 5-star reviews for all Vine members : {per_fivestar_vine_reviews}%.')

In [ ]:
# The total number of reviews for non-Vine members
total_non_vine_reviews = unpaid_df.count()

# The number of 5-star reviews for all non-Vine members
fivestar_non_vine_reviews = unpaid_df.filter(df['star_rating'] == 5).count()

#  percentage of 5-star reviews  for all non-Vine members
per_fivestar_non_vine_reviews = fivestar_non_vine_reviews/total_non_vine_reviews * 100
print(f'The total number of reviews for non-Vine members is : {total_non_vine_reviews}.\n \
The number of 5-star reviews for all non-Vine members: {fivestar_non_vine_reviews}.\n  \
The percentage of 5-star reviews for all non-Vine members : {per_fivestar_non_vine_reviews}%.')

In [ ]:
# Additional analysis
# Total ratio of five star versus non five star reviews for Vine members
non_fivestar_vine_reviews = paid_df.filter(df['star_rating'] != 5).count()

ratio_fivestar_non_fivestar_vine = fivestar_vine_reviews/non_fivestar_vine_reviews*100

# Total ratio of five star versus non five star reviews for non-Vine members
non_fivestar_non_vine_reviews = unpaid_df.filter(df['star_rating'] != 5).count()

ratio_fivestar_non_fivestar_non_vine = fivestar_non_vine_reviews/non_fivestar_non_vine_reviews *100 

print(f'The ratio of fivestar reviews to non fivestar reviews for Vine Members is: {ratio_fivestar_non_fivestar_vine}%. \n \
The ratio of fivestar reviews to non fivestar reviews for non-Vine Members is: {ratio_fivestar_non_fivestar_non_vine}%.\n')